In [ ]:
class DICcomp:
    '''
    class DICcomp
    
    Compare DIC to Glodap
    
    -use layerwise = True to compare a set of depth
    -use depth_limit to specify maximum depth for mean-over-depth comparison
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 GLODAPvar='TCO2_mmol',
                 mapproj='pc',
                 cmap = 'viridis',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 uplow=[0, 100],
                 cmap_extension='max',
                 verbose=False,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 get_overview = False,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.GLODAPvar = GLODAPvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.uplow = uplow
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_GLODAP import GLODAPdata
        from Py_f2recom_toolbox import plt_Taylor_norm
        from Py_f2recom_toolbox import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        #print('Processing {0}'.format(self.resultpath))
        self.mapproj = pf.get_proj(self.mapproj)
        years = np.arange(self.fyear, self.lyear+1,1)

        # check variables
        #NCfesom = self.resultpath + '/DFe.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom

        labelfesom = 'FESOM'
        unitfesom = 'DIC [mmol C m$^{-3}$]' # equals to mumol/L

        # load FESOM data -------------------------------------------------------------------------------------
        DICfesom = pf.get_data(resultpath, "DIC", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent = True)
        
        unitglodap = 'DIC [mmol C m$^{-3}$]'
        labelglodap = 'GLODAP'
        
        # interpolate Glodap data -------------------------------------------------------------------------------------
        GLODAP_input = GLODAPdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.GLODAPvar, get_overview=False)
        glodap_int = GLODAP_input.glodap_int   

        # apply sea mask to Glodap as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        glodap_int_ma = np.copy(glodap_int)
        glodap_int_ma[DICfesom == 0] = 0

        # plot Glodap and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)
                
            for i in np.arange(len(depth_array)-1):
                if depth_array[i+1] < -np.max(GLODAP_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))
                
                uplow = [depth_array[i], depth_array[i+1]]


                DICfesom_mean = pf.layermean_data(DICfesom, mesh, uplow = uplow)
                glodap_int_ma_mean = pf.layermean_data(glodap_int_ma, mesh, uplow = uplow)
                
                if(self.verbose):
                    print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nWOA mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                        np.nanmean(DICfesom_mean),np.nanstd(DICfesom_mean),np.nanmin(DICfesom_mean),np.nanmax(DICfesom_mean),
                        np.nanmean(glodap_int_ma_mean),np.nanstd(glodap_int_ma_mean),np.nanmin(glodap_int_ma_mean),np.nanmax(glodap_int_ma_mean)))
                
                if plotting:
                    fig = plt.figure(figsize=(15,12), constrained_layout=False)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(1600,2300,10)
                    f1 = pf.subplot(mesh, fig, m1, [DICfesom_mean],
                                levels = levels,
                                units=unitglodap, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelfesom+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, fig, m2, [glodap_int_ma_mean], 
                                levels = levels,
                                units=unitglodap, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelglodap+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-150,160,10)
                    f3 = pf.subplot(mesh, fig, m3, [DICfesom_mean-glodap_int_ma_mean], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitglodap, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - GLODAP ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )
                    
                    m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                                size=30, weight='bold')
                    m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                                size=30, weight='bold')
                    m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                                size=30, weight='bold')

                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    plt.show(block=False)
                    
                    
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(glodap_int)): print('WARNING: The interpolated GLODAP field contains NaNs at depth')
                    if np.isnan(np.min(DICfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Normalized Taylor Diagram for DIC'
                    plt_Taylor_comp(dic_int_ma,DICfesom,mask=True,title=title, depth_array=depth_array, mesh=mesh)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_layerwise.png', 
                                        dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_layerwise.pdf', 
                                        bbox_inches='tight')
                    plt.show(block=False)
                    
                if output:
                    #self.fesom.{str(d)} = DICfesom_mean
                    #self.glodap.{str(d)}   = glodap_int_ma_mean
                    print('returning layerwise output not supported yet')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            if uplow[1] < -np.max(GLODAP_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))

            DICfesom_mean = pf.layermean_data(DICfesom, mesh, uplow = uplow)
            glodap_int_ma_mean = pf.layermean_data(glodap_int_ma, mesh, uplow = uplow)

            if(self.verbose):
                print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nPISCES mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                np.nanmean(DICfesom_mean),np.nanstd(DICfesom_mean),np.nanmin(DICfesom_mean),np.nanmax(DICfesom_mean),
                np.nanmean(glodap_int_ma_mean),np.nanstd(glodap_int_ma_mean),np.nanmin(glodap_int_ma_mean),np.nanmax(glodap_int_ma_mean)))
            
            if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(1600,2300,10)
                f1 = pf.subplot(mesh, fig, m1, [DICfesom_mean],
                            levels = levels,
                            units=unitglodap, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [glodap_int_ma_mean], 
                            levels = levels,
                            units=unitglodap, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelglodap+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-150,160,10)
                f3 = pf.subplot(mesh, fig, m3, [DICfesom_mean-glodap_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitglodap, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - GLODAP'+' ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                                size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                                size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                                size=30, weight='bold')
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'DIC_GLODAP'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
                
            if Taylor:
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(glodap_int_ma_mean)): print('WARNING: The interpolated GLODAP field contains NaNs at depth')
                if np.isnan(np.min(DICfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


                title = 'Taylor Diagram for DIC \n(mean over depth, max = {0}-{1}m)'.format(uplow[0],uplow[1]),
                plt_Taylor_norm(glodap_int_ma_mean,DICfesom_mean,mask=True,title=title)
                
                
                # fig export  
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'DIC_Glodap_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'DIC_Glodap_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
                
            if output:
                    self.fesom = DICfesom_mean
                    self.glodap   = glodap_int_ma_mean